In [13]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import random

### The Mandelbrot set
The Mandelbrot set is a two-dimensional set that is defined in the complex plane as the complex numbers $c$ for which the function $f_c(z) = z^2 + c $ does not diverge to infinity when iterated starting at $z=0$.

Interesting properties:
- A point c belongs to the Mandelbrot set iff $|z| \leq 2$ for all $n \geq 0$

### Representing complex numbers

To represent the complex numbers, I will use an array of two float, where the first element represents the real part and the second element the imaginary part.

In [14]:
def square(x):
    result = [0, 0]

    result[0] = x[0]**2 - x[1]**2
    result[1] = 2 * x[0] * x[1]

    return result


def compute(c, z):
    """Compute z^2 + c"""
    z_square = square(z)
    return [c[0] + z_square[0] , c[1] + z_square[1]]


def isInMandelbrotSet(c, z=[0, 0], max_iter=1000, n=0) -> bool:
    if z[0]**2 + z[1]**2 > 4:
        return False
    
    if n==max_iter:
        return True

    return isInMandelbrotSet(c, compute(c, z), max_iter, n+1)


In [15]:
print(f"Is [-1,0] in the set?: {isInMandelbrotSet([-1, 0])}")  # should be true
print(f"Is [1,0] in the set?: {isInMandelbrotSet([1, 0])}")  # should be false

Is [-1,0] in the set?: True
Is [1,0] in the set?: False


### Creating a dataset

In [16]:
# Define the range for the real and imaginary parts
real_range = np.linspace(-2, 1, 50)
imag_range = np.linspace(-1.5, 1.5, 50)

data = []
for a in real_range:
    for b in imag_range:
        c = [a, b]
        is_in_set = isInMandelbrotSet(c)
        data.append({
            'real': a,
            'imag': b,
            'in_mandelbrot_set': is_in_set
        })

df = pd.DataFrame(data)

In [17]:
df

,real,imag,in_mandelbrot_set
0,-2.0,-1.500000,False
1,-2.0,-1.438776,False
2,-2.0,-1.377551,False
3,-2.0,-1.316327,False
4,-2.0,-1.255102,False
...,...,...,...
2495,1.0,1.255102,False
2496,1.0,1.316327,False
2497,1.0,1.377551,False
2498,1.0,1.438776,False


In [18]:
df['in_mandelbrot_set'].value_counts()

in_mandelbrot_set
False    2094
True      406
Name: count, dtype: int64

In [19]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=["in_mandelbrot_set"])
y = df["in_mandelbrot_set"]

# Using stratified split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42, shuffle=True
)


### Neural Network

In [20]:
class NeuralNet(nn.Module):
    def __init__(self, num_hidden_layers=3):
        super().__init__()
        
        layers = [
            nn.Linear(2, 32),
            nn.ReLU(),
        ]

        for _ in range(num_hidden_layers):
            layers.append(
                nn.Linear(32, 32)
            )
            layers.append(
                nn.ReLU()
            )
        
        # Output
        layers.append(
            nn.Linear(32, 1)
        )
        self.sigmoid_layer = nn.Sigmoid()
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        x = self.network(x)
        return self.sigmoid_layer(x)

In [21]:
mandelbrot_net = NeuralNet()

### Training Loop

In [22]:
from torch.utils.data import TensorDataset, DataLoader

NUM_EPOCH = 20
LEARNING_RATE = 0.0001
BATCH_SIZE = 16

criterion = nn.BCELoss()
optimizer = optim.Adam(mandelbrot_net.parameters(), lr=LEARNING_RATE)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1) # Convert target to float and unsqueeze for BCEWithLogitsLoss

# Create a TensorDataset and DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Training loop
for epoch in range(NUM_EPOCH):
    for batch_X, batch_y in train_loader:
        # Forward pass
        outputs = mandelbrot_net(batch_X)
        loss = criterion(outputs, batch_y)

        # Backward pass and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")



Epoch: 1, Loss: 0.589841365814209
Epoch: 2, Loss: 0.4113062918186188
Epoch: 3, Loss: 0.1962919384241104
Epoch: 4, Loss: 0.3264751136302948
Epoch: 5, Loss: 0.31305480003356934
Epoch: 6, Loss: 0.3673720061779022
Epoch: 7, Loss: 0.24531084299087524
Epoch: 8, Loss: 0.3464818596839905
Epoch: 9, Loss: 0.36748579144477844
Epoch: 10, Loss: 0.17475436627864838
Epoch: 11, Loss: 0.2887818515300751
Epoch: 12, Loss: 0.1255597174167633
Epoch: 13, Loss: 0.2110673487186432
Epoch: 14, Loss: 0.16151145100593567
Epoch: 15, Loss: 0.18313631415367126
Epoch: 16, Loss: 0.01199581753462553
Epoch: 17, Loss: 0.09213060885667801
Epoch: 18, Loss: 0.12210164964199066
Epoch: 19, Loss: 0.12487905472517014
Epoch: 20, Loss: 0.029407594352960587


### Testing Loop

In [23]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Convert data to Pytorch tensors
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)

# Create a TensorDataset and DataLoader
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Evaluation mode
mandelbrot_net.eval() 

with torch.no_grad():
    y_pred_tensor = mandelbrot_net(X_test_tensor)
    
    # Convert probabilities to binary predictions (0 or 1)
    y_pred = (y_pred_tensor > 0.5).float() 

    # Calculate metrics
    accuracy = accuracy_score(y_test_tensor.numpy(), y_pred.numpy())
    print(f"Test Accuracy: {accuracy:.4f}\n")
    
    print("Classification Report:")
    print(classification_report(y_test_tensor.numpy(), y_pred.numpy()))
    
    print("Confusion Matrix:")
    print(confusion_matrix(y_test_tensor.numpy(), y_pred.numpy()))

Test Accuracy: 0.9640

Classification Report:
              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98       419
         1.0       0.92      0.85      0.88        81

    accuracy                           0.96       500
   macro avg       0.95      0.92      0.93       500
weighted avg       0.96      0.96      0.96       500

Confusion Matrix:
[[413   6]
 [ 12  69]]
